In [2]:
import pandas as pd
import numpy as np
import sklearn
import math

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from sklearn.metrics import classification_report, accuracy_score

In [3]:
# Считывание csv-файлов
train = pd.read_csv('csv/train.csv')
test = pd.read_csv('csv/test.csv')
gender_submission = pd.read_csv('csv/gender_submission.csv')

# Объеденение датасетов
test_merge = pd.merge(test, gender_submission, how='left', on='PassengerId')
df = pd.concat([test_merge, train], ignore_index=True)

# Обработка Pclass
df['Pclass'] = df['Pclass'].apply(lambda x: 3 - x + 1)

# Обработка Sex
df['sex_rank'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1)

# Обработка Cabin
df['Cabin_class'] = df['Cabin'].apply(lambda x: np.nan if type(x) != str and np.isnan(x) else x[0])
df.iloc[[479, 1247], 10] = 'S'

# Обработка Embarked
df['embarked_s'] = df['Embarked'].apply(lambda x: 1 if x == 'S' else 0)
df['embarked_c'] = df['Embarked'].apply(lambda x: 1 if x == 'C' else 0)
df['embarked_q'] = df['Embarked'].apply(lambda x: 1 if x == 'Q' else 0)

# Итоговый датасет
df = df[['Pclass', 'sex_rank', 'Age', 'SibSp', 'Parch', 'Fare', 'embarked_s', 'embarked_c', 'embarked_q', 'Survived']]

# Обработка пропусков в Age и Fare 
age_median = df['Age'].median()
df['Age'] = df['Age'].apply(lambda x: age_median if np.isnan(x) else x)

# Удаление дублей
df.drop_duplicates(inplace=True, ignore_index=True)
df.iloc[145, 5] = df[df['Pclass'] == 1]['Fare'].mean()

### Краткий гайд по методу

### PCA in Sklearn

In [4]:
x, y = df.iloc[:, :-1], df.iloc[:, -1]

In [9]:
pca = PCA(n_components=2)
len(pca.fit_transform(x))

1100

In [7]:
print(pca.explained_variance_ratio_)

[0.94188914 0.05725701]
